In [1]:
import pandas as pd 
import streamlit as st
import functions as fc
import home
import sgs
import plotly.graph_objects as go

In [8]:
# Lista de códigos da API SGS BACEN
cod_list = pd.read_csv('base_csv/cod_selic.csv', sep=';').set_index('codigo')
cod_list

,serie,lista
codigo,,
11,SELIC (Taxa de Juros diária),11-SELIC (Taxa de Juros diária)
1178,SELIC (Taxa de Juros Anual),1178-SELIC (Taxa de Juros Anual)
432,SELIC (Taxa de juros meta),432-SELIC (Taxa de juros meta)
4390,SELIC (Taxa de Juros acumulado Mês),4390-SELIC (Taxa de Juros acumulado Mês)


In [9]:
def df_sgs(lista_sgs,ano_inicio, ano_fim, name):
    df = pd.DataFrame()
    for i in lista_sgs:
        data_inicio = ano_inicio
        data_fim = ano_fim
        split = i.split(sep='-')
        df1 = sgs.time_serie(split[0], data_inicio, data_fim,True)
        df1 = pd.DataFrame(df1)
        df1 = df1.rename(columns={f'{split[0]}':f'{split[1]}'})
        df = pd.concat([df,df1], axis=1)
    return df.to_csv(f'base_csv/base_{name}.csv')

In [11]:
df_sgs(cod_list['lista'], '01/01/1980', '01/04/2021', 'selic')

In [14]:
df = pd.read_csv('base_csv/base_selic.csv', encoding='UTF-8', sep=',', index_col=0)
df1 = pd.DataFrame(df['SELIC (Taxa de Juros acumulado Mês)'])
df1

,SELIC (Taxa de Juros acumulado Mês)
1986-06-01,1.27
1986-06-04,NaN
1986-06-05,NaN
1986-06-06,NaN
1986-06-09,NaN
...,...
2021-03-28,NaN
2021-03-29,NaN
2021-03-30,NaN
2021-03-31,NaN


In [6]:
def graf_plotly(data_frame,ano_inicio, ano_fim, titulo):
    fig = go.Figure()
    fig.update_layout(
    title= f'{titulo}',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        title= '%',
        showgrid=False,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white',
    legend= dict(
        font=dict(
            family='Arial',
            size=12))
    )
    count = 0
    for i in data_frame.columns:
        if count < 2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="markers+lines", visible=True))
        elif count == 2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="lines", visible=True))
        elif count >2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="lines", visible='legendonly'))
    return lines

In [7]:
graf_plotly(df,f'01/01/1980', f'01/01/2021', 'IPCA - % Variação Mensal Total')